In [1]:
#standard ds imports
import pandas as pd
import numpy as np
#viz and stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# .py imports
from env import user, pwd,host
import env
import wranglerer as wr
import modeling as md
import os
#sklearn imports
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from sklearn.metrics import mean_squared_error,explained_variance_score

from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
#CATboost imports
from catboost import CatBoostRegressor

## ACQUIRE

In [2]:
# ACQUIRE/PREP
df = wr.merge_wines_df()

local version found!
local version found!


## SPLIT

In [3]:
# SPLIT
X_train, y_train, X_validate, y_validate, X_test, y_test = wr.train_validate_test(df,'quality')

In [4]:
X_train.shape, y_train.shape, X_validate.shape, y_validate.shape, X_test.shape, y_test.shape

((3637, 12), (3637,), (1560, 12), (1560,), (1300, 12), (1300,))

In [5]:
X_train.columns

Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'ph', 'sulphates', 'alcohol', 'wine_color'],
      dtype='object')

## SCALE (MinMax)

In [6]:
# SCALE THE NUMERICAL DRIVERS
cols = X_train.columns
X_train_scaled, X_validate_scaled, X_test_scaled = wr.min_max_scale(X_train,X_validate,X_test,cols)

## EST BASELINE (mean)

In [7]:
# ESTABLISH BASELINE:
# convert y_targets into dataframe so that we can append additional columns to it
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)
y_test = pd.DataFrame(y_test)

# Predict target_pred_mean-Baseline
target_pred_mean = y_train.quality.mean()
y_train['target_pred_mean'] = round(target_pred_mean,0)
y_validate['target_pred_mean'] = target_pred_mean

# 2. compute target_pred_median
target_pred_median = y_train.quality.median()
y_train['target_pred_median'] = target_pred_median
y_validate['target_pred_median'] = target_pred_median

#Calculate RMSE for Baseline:
rmse_train_mean = round(mean_squared_error(y_train.quality,
                                y_train.target_pred_mean) ** (.5),1)
rmse_validate_mean = round(mean_squared_error(y_validate.quality, y_validate.target_pred_mean) ** (0.5),1)

rmse_train_med = mean_squared_error(y_train.quality, y_train.target_pred_median) ** .5
rmse_validate_med = mean_squared_error(y_validate.quality, y_validate.target_pred_median) ** (0.5)

print(f"""RMSE using Mean
Train/In-Sample: {round(rmse_train_mean, 1)} 
Validate/Out-of-Sample: {round(rmse_validate_mean, 1)}""")
print(f"""RMSE using Median
Train/In-Sample: {round(rmse_train_med, 2)} 
Validate/Out-of-Sample: {round(rmse_validate_med, 2)}""")

# USE MEAN as BASELINE

RMSE using Mean
Train/In-Sample: 0.9 
Validate/Out-of-Sample: 0.9
RMSE using Median
Train/In-Sample: 0.89 
Validate/Out-of-Sample: 0.91


In [8]:
# Drop Median...remove unnecessary clutter
y_train = y_train.drop(columns='target_pred_median')

## K-BEST / RFE

In [9]:
# K-BEST results:
# make the thing
kbest = SelectKBest(f_regression,k=4)
# fit the thing
_ = kbest.fit(X_train_scaled,y_train.quality)
# statistical f-value:
kbest.scores_
X_train_scaled.columns[kbest.get_support()]

Index(['volatile_acidity', 'chlorides', 'density', 'alcohol'], dtype='object')

In [10]:
y_train

,quality,target_pred_mean
3737,5,6.0
19,6,6.0
2200,6,6.0
4139,5,6.0
501,7,6.0
...,...,...
4918,5,6.0
3542,5,6.0
1884,5,6.0
2206,5,6.0


In [11]:
# RFE results:
# establish a model for RFE to use
model = LinearRegression()
# make an RFE thing
rfe = RFE(model, n_features_to_select=4)
# fit the RFE thing
rfe.fit(X_train_scaled,y_train.quality)
rfe_df = pd.DataFrame({
    'rfe_ranking':rfe.ranking_
}, index=X_train_scaled.columns)
rfe_df.sort_values('rfe_ranking')

,rfe_ranking
volatile_acidity,1
residual_sugar,1
sulphates,1
alcohol,1
free_sulfur_dioxide,2
total_sulfur_dioxide,3
density,4
fixed_acidity,5
ph,6
chlorides,7


## METRIC_DF

In [13]:
# build a quick df to compare results...will append to this df as needed
metric_df = pd.DataFrame(data=[
    {'model': 'mean_baseline',
     'RMSE_train': rmse_train_mean,
     'RMSE_validate': rmse_validate_mean,
     'R2_validate': explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_mean)
    }
]
)
metric_df

,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.9,0.9,0.0


## MODEL 1 OLS
- volatile_acidity, alcohol, density

In [14]:
#define features for MODEL 1:
model_1_feats = ['alcohol',
                'volatile_acidity',
                'density']
X_train_1 = X_train[model_1_feats]
X_validate_1 = X_validate[model_1_feats]
X_test_1 = X_test[model_1_feats]

# MAKE THE THING: create the model object
lm = LinearRegression()
# FIT THE THING: fit the model to training data
OLSmodel = lm.fit(X_train_1, y_train.quality)
# USE THE THING: make a prediction
y_train['target_pred_lm'] = lm.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_lm)**(1/2)
# predict validate
y_validate['target_pred_lm'] = lm.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_lm)**(1/2)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.7436609095570793 
Validation/Out-of-Sample: 0.7557847149398661


In [15]:
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'OLS Model 1 (base feats)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_lm),4)
    }, ignore_index=True)
metric_df

/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_1917/1027816449.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782


## MODEL 1 OLS
- add wine_color

In [16]:
#define features for MODEL 1:
model_1_feats = ['alcohol',
                'volatile_acidity',
                'density',
                'wine_color']
X_train_1 = X_train[model_1_feats]
X_validate_1 = X_validate[model_1_feats]
X_test_1 = X_test[model_1_feats]

# MAKE THE THING: create the model object
lm = LinearRegression()
# FIT THE THING: fit the model to training data
OLSmodel = lm.fit(X_train_1, y_train.quality)
# USE THE THING: make a prediction
y_train['target_pred_lm'] = lm.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_lm)**(1/2)
# predict validate
y_validate['target_pred_lm'] = lm.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_lm)**(1/2)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.7431262818804543 
Validation/Out-of-Sample: 0.7536053512471359


In [17]:
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'OLS Model 1 (base + color)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_lm),4)
    }, ignore_index=True)
metric_df

/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_1917/1224112532.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823


## MODEL 1 OLS
- add chlorides

In [18]:
#define features for MODEL 1:
model_1_feats = ['alcohol',
                'volatile_acidity',
                'density',
                'wine_color',
                'chlorides']
X_train_1 = X_train[model_1_feats]
X_validate_1 = X_validate[model_1_feats]
X_test_1 = X_test[model_1_feats]

# MAKE THE THING: create the model object
lm = LinearRegression()
# FIT THE THING: fit the model to training data
OLSmodel = lm.fit(X_train_1, y_train.quality)
# USE THE THING: make a prediction
y_train['target_pred_lm'] = lm.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_lm)**(1/2)
# predict validate
y_validate['target_pred_lm'] = lm.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_lm)**(1/2)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.7425498029952226 
Validation/Out-of-Sample: 0.7529423946142223


In [19]:
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'OLS Model 1 (+ chlorides)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_lm),4)
    }, ignore_index=True)
metric_df

/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_1917/858112617.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823
3,OLS Model 1 (+ chlorides),0.74,0.75,0.2836


## MODEL 1 OLS
- add sulphates

In [20]:
#define features for MODEL 1:
model_1_feats = ['alcohol',
                'volatile_acidity',
                'density',
                'wine_color',
                'chlorides',
                'sulphates']
X_train_1 = X_train[model_1_feats]
X_validate_1 = X_validate[model_1_feats]
X_test_1 = X_test[model_1_feats]

# MAKE THE THING: create the model object
lm = LinearRegression()
# FIT THE THING: fit the model to training data
OLSmodel = lm.fit(X_train_1, y_train.quality)
# USE THE THING: make a prediction
y_train['target_pred_lm'] = lm.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_lm)**(1/2)
# predict validate
y_validate['target_pred_lm'] = lm.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_lm)**(1/2)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.7401442039707741 
Validation/Out-of-Sample: 0.7483438899735064


In [21]:
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'OLS Model 1 (+ sulphates)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_lm),4)
    }, ignore_index=True)
metric_df

/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_1917/1561224617.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823
3,OLS Model 1 (+ chlorides),0.74,0.75,0.2836
4,OLS Model 1 (+ sulphates),0.74,0.75,0.2923


In [22]:
#define features for MODEL 1:
model_1_feats = ['alcohol',
                'volatile_acidity',
                'density',
                'wine_color',
                'chlorides',
                'sulphates']
X_train_1 = X_train_scaled[model_1_feats]
X_validate_1 = X_validate_scaled[model_1_feats]
X_test_1 = X_test_scaled[model_1_feats]

# MAKE THE THING: create the model object
lm = LinearRegression()
# FIT THE THING: fit the model to training data
OLSmodel = lm.fit(X_train_1, y_train.quality)
# USE THE THING: make a prediction
y_train['target_pred_lm'] = lm.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_lm)**(1/2)
# predict validate
y_validate['target_pred_lm'] = lm.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_lm)**(1/2)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.7401442039707741 
Validation/Out-of-Sample: 0.7483438899735064


In [23]:
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'OLS Model 1 Scaled (+ sulphates)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_lm),4)
    }, ignore_index=True)
metric_df

/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_1917/178744154.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823
3,OLS Model 1 (+ chlorides),0.74,0.75,0.2836
4,OLS Model 1 (+ sulphates),0.74,0.75,0.2923
5,OLS Model 1 Scaled (+ sulphates),0.74,0.75,0.2923


## MODEL 2 DTR maxDepth=7
- volatile_acidity, alcohol, density

In [24]:
model_2_feats = ['alcohol',
                'volatile_acidity',
                'density',
                'wine_color',
                'chlorides',
                'sulphates',
                'citric_acid']
X_train_1 = X_train[model_2_feats]
X_validate_1 = X_validate[model_2_feats]
X_test_1 = X_test[model_2_feats]

# Create and fit the thing
DTRp = DecisionTreeRegressor(max_depth=5)
DTRp.fit(X_train_1,y_train.quality)
y_train['target_pred_DTRp'] = DTRp.predict(X_train_1)
DTRp.predict(X_train_1)
# evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_DTRp)**(1/2)
# predict validate
y_validate['target_pred_DTRp'] = DTRp.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_DTRp)**(1/2)
r2 = r2_score(y_train.quality,y_train.target_pred_DTRp)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}
Train R2 Score: {r2}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.700572389727326 
Validation/Out-of-Sample: 0.7480412396615373
Train R2 Score: 0.34597177670463775


In [25]:
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'DTR: maxD=5 (+ citric)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_DTRp),4)
    }, ignore_index=True)
metric_df

/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_1917/2999740504.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823
3,OLS Model 1 (+ chlorides),0.74,0.75,0.2836
4,OLS Model 1 (+ sulphates),0.74,0.75,0.2923
5,OLS Model 1 Scaled (+ sulphates),0.74,0.75,0.2923
6,DTR: maxD=5 (+ citric),0.70,0.75,0.2932


## MODEL 3 RFR n_estimators=55, maxDepth= 5
- volatile_acidity, alcohol, density

In [32]:
#model_3_feats = ['alcohol',
#                 'volatile_acidity',
#                 'density',
#                 'wine_color',
#                 'chlorides',
#                 'sulphates',
#                 'citric_acid']
X_train_1 = X_train
X_validate_1 = X_validate
X_test_1 = X_test

# Create and fit the thing
RFRp = RandomForestRegressor(n_estimators=1001, max_depth=8,random_state=2013)
RFRp.fit(X_train_1,y_train.quality)
y_train['target_pred_RFRp'] = RFRp.predict(X_train_1)
RFRp.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_RFRp)**(1/2)
#REPEAT STEPS 2-3
# predict validate
y_validate['target_pred_RFRp'] = RFRp.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_RFRp)**(1/2)
r2 = r2_score(y_train.quality,y_train.target_pred_RFRp)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}
Train R2 Score: {r2}""")
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'RFR ALL FEATS est=1001, maxD=8',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_RFRp),4)
    }, ignore_index=True)

metric_df

RMSE for OLS using LinearRegression
Training/In-Sample:  0.5465231946481608 
Validation/Out-of-Sample: 0.6753807567880562
Train R2 Score: 0.6019774738089717


/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_1917/1450813207.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823
3,OLS Model 1 (+ chlorides),0.74,0.75,0.2836
4,OLS Model 1 (+ sulphates),0.74,0.75,0.2923
5,OLS Model 1 Scaled (+ sulphates),0.74,0.75,0.2923
6,DTR: maxD=5 (+ citric),0.70,0.75,0.2932
7,"RFR est=55, maxD=5",0.49,0.69,0.3969
8,"RFR ALL FEATS est=55, maxD=5",0.46,0.66,0.4499
9,"RFR ALL FEATS est=301, maxD=5",0.45,0.66,0.4570


## CATBOOST
- all feats

In [40]:
model_4_feats = ['alcohol',
                'volatile_acidity',
                'density',
                'wine_color',
                'chlorides',
                'sulphates',
                'citric_acid']
X_train_1 = X_train[model_4_feats]
X_validate_1 = X_validate[model_4_feats]
X_test_1 = X_test[model_4_feats]

# Create and fit the thing
CATb = CatBoostRegressor(verbose=False,depth=10)
CATb.fit(X_train_1,y_train.quality)
y_train['target_pred_CATb'] = CATb.predict(X_train_1)
CATb.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_CATb)**(1/2)
#REPEAT STEPS 2-3
# predict validate
y_validate['target_pred_CATb'] = CATb.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_CATb)**(1/2)
r2 = r2_score(y_train.quality,y_train.target_pred_CATb)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}
Train R2 Score: {r2}""")
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'CATb maxD=10 (+citric)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_CATb),4)
    }, ignore_index=True)

metric_df

RMSE for OLS using LinearRegression
Training/In-Sample:  0.28603022213235957 
Validation/Out-of-Sample: 0.6713439992428252
Train R2 Score: 0.8909779605663152


/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_1917/576454244.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823
3,OLS Model 1 (+ chlorides),0.74,0.75,0.2836
4,OLS Model 1 (+ sulphates),0.74,0.75,0.2923
5,OLS Model 1 Scaled (+ sulphates),0.74,0.75,0.2923
6,DTR: maxD=5 (+ citric),0.70,0.75,0.2932
7,"RFR est=55, maxD=5",0.49,0.69,0.3969
8,"RFR ALL FEATS est=55, maxD=5",0.46,0.66,0.4499
9,"RFR ALL FEATS est=301, maxD=5",0.45,0.66,0.4570
